In [18]:
### This function is to append planar images from two different folders 

import os
import json
import shutil

def append_folder(src_folder, dest_folder, start_idx):
    """
    Append the contents of the source folder to the destination folder.

    src_folder: Path to the source folder.
    dest_folder: Path to the destination folder.
    start_idx: Starting index for renaming files in the destination folder.
    """
    
    # Ensure the destination folder exists
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    # List all RGB images in the source folder
    image_files = sorted([f for f in os.listdir(src_folder) if f.endswith(".rgb.jpg")])
    
    for image_file in image_files:
        # Generate new filename based on the starting index
        new_image_name = f"{str(start_idx).zfill(6)}.rgb.jpg"
        
        # Copy and rename the image to the destination folder
        shutil.copy(os.path.join(src_folder, image_file), os.path.join(dest_folder, new_image_name))
        
        # Load the corresponding JSON annotation from the source folder
        json_file = image_file.replace('.rgb.jpg', '.json')
        with open(os.path.join(src_folder, json_file), 'r') as f:
            annotation = json.load(f)

        # Update the id and image_rgb fields in the JSON annotation
        annotation["id"] = start_idx
        annotation["image_rgb"] = new_image_name
        
        # Save the updated JSON annotation to the destination folder
        with open(os.path.join(dest_folder, new_image_name.replace('.rgb.jpg', '.json')), 'w') as f:
            json.dump(annotation, f)

        # Increment the starting index for the next file
        start_idx += 1

    print(f"Appended files from {src_folder} to {dest_folder}.")


# Example usage:
src_folder = "/home/jc-merlab/Pictures/Data/source_folder_occlusion/"
dest_folder = "/home/jc-merlab/Pictures/Data/occ_sim_dataset/"
start_idx = 5324
append_folder(src_folder, dest_folder, start_idx)

Appended files from /home/jc-merlab/Pictures/Data/source_folder_occlusion/ to /home/jc-merlab/Pictures/Data/occ_sim_dataset/.


In [2]:
import os
import cv2
import json

def visualize_keypoints(folder_path):
    """
    Visualize the keypoints for each image in the given folder.

    folder_path: Path to the folder containing images and their JSON annotations.
    """
    
    # List all RGB images in the folder
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".rgb.jpg")])
    
    for image_file in image_files:
        # Load the image
        image_path = os.path.join(folder_path, image_file)
        image = cv2.imread(image_path)
        
        # Load the corresponding JSON annotation
        json_file = image_file.replace('.rgb.jpg', '.json')
        with open(os.path.join(folder_path, json_file), 'r') as f:
            annotation = json.load(f)

        # Draw keypoints on the image
        for kp_set in annotation['keypoints']:
            for kp in kp_set:
                x, y, visibility = kp
                if visibility == 1:  # Only draw visible keypoints
                    cv2.circle(image, (int(x), int(y)), 5, (0, 255, 0), -1)

        # Display the image with keypoints
        cv2.imshow('KeyPoints Visualization', image)
        cv2.waitKey(0)

    cv2.destroyAllWindows()


# Example usage:
folder_path = "/home/jc-merlab/Pictures/Data/2023-10-20/"
visualize_keypoints(folder_path)

FileNotFoundError: [Errno 2] No such file or directory: '/home/jc-merlab/Pictures/Data/2023-10-20/'

In [16]:
import random
import numpy as np

# def apply_rectangle_occlusion(image, x, y, occlusion_size, color=None):
#     if color is None:
#         color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
#     image[y:y+occlusion_size, x:x+occlusion_size, :] = color
#     return image

def apply_rectangle_occlusion(image, x, y, width, height, color=None):
    if color is None:
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

    # Create a blank rectangle of desired dimensions
    rectangle = np.ones((height, width, 3), dtype=image.dtype) * np.array(color, dtype=image.dtype)

    # Randomly rotate the rectangle
    angle = random.randint(-45, 45)  # 45-degree rotation in either direction
    M = cv2.getRotationMatrix2D((width//2, height//2), angle, 1)
    rotated_rectangle = cv2.warpAffine(rectangle, M, (width, height))

    # Place the rotated rectangle on the image
    image[y:y+height, x:x+width, :] = rotated_rectangle
    return image

def apply_circle_occlusion(image, x, y, occlusion_size, color=None):
    if color is None:
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    center = (x + occlusion_size//2, y + occlusion_size//2)
    cv2.circle(image, center, occlusion_size//2, color, -1)
    return image

def apply_ellipse_occlusion(image, x, y, occlusion_size, color=None):
    if color is None:
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    center = (x + occlusion_size//2, y + occlusion_size//2)
    axes = (occlusion_size//2, random.randint(occlusion_size//4, occlusion_size//2))
    angle = random.randint(0, 180)
    cv2.ellipse(image, center, axes, angle, 0, 360, color, -1)
    return image

# def random_occlusion(image, keypoints, max_occlusions=3, min_size=60, max_size=100):
#     height, width, _ = image.shape
#     num_occlusions = random.randint(1, max_occlusions)

#     for _ in range(num_occlusions):
#         # Randomly select a keypoint to occlude
#         kp_set = random.choice(keypoints[:6])
#         kp = random.choice(kp_set)[:2]  # Ignore visibility flag

#         # Random size for the occlusion
#         occlusion_size = random.randint(min_size, max_size)

#         # Center the occlusion on the keypoint
#         x = int(kp[0] - occlusion_size/2)
#         y = int(kp[1] - occlusion_size/2)

#         # Ensure the occlusion is within image boundaries
#         x = max(0, min(x, width - occlusion_size))
#         y = max(0, min(y, height - occlusion_size))

#         # Random shape of occlusion
#         shapes = ['rectangle', 'circle', 'ellipse']
#         shape = random.choice(shapes)
#         if shape == 'rectangle':
#             image = apply_rectangle_occlusion(image, x, y, occlusion_size)
#         elif shape == 'circle':
#             image = apply_circle_occlusion(image, x, y, occlusion_size)
#         elif shape == 'ellipse':
#             image = apply_ellipse_occlusion(image, x, y, occlusion_size)

#     return image, keypoints


def random_occlusion(image, keypoints, max_occlusions=3, min_size=30, max_size=60):
    height, width, _ = image.shape
    num_occlusions = random.randint(1, max_occlusions)

    occluded_keypoints = []  # Keep track of which keypoints are occluded

    for _ in range(num_occlusions):
        # Randomly select a keypoint to occlude
        kp_index = random.randint(0, len(keypoints[:6]) - 1)
        kp_set = keypoints[kp_index]
        kp = random.choice(kp_set)[:2]  # Ignore visibility flag

        # Mark this keypoint as occluded
        occluded_keypoints.append((kp_index, kp))

        # Random size for the occlusion
        occlusion_width = random.randint(min_size, max_size)
        occlusion_height = random.randint(min_size, max_size)

        # Center the occlusion on the keypoint
        x = int(kp[0] - occlusion_width / 2)
        y = int(kp[1] - occlusion_height / 2)

        # Ensure the occlusion is within image boundaries
        x = max(0, min(x, width - occlusion_width))
        y = max(0, min(y, height - occlusion_height))

        # Random shape of occlusion
        shapes = ['rectangle', 'circle', 'ellipse']
        shape = random.choice(shapes)
        if shape == 'rectangle':
            image = apply_rectangle_occlusion(image, x, y, occlusion_width, occlusion_height)
        elif shape == 'circle':
            image = apply_circle_occlusion(image, x, y, max(occlusion_width, occlusion_height))
        elif shape == 'ellipse':
            image = apply_ellipse_occlusion(image, x, y, max(occlusion_width, occlusion_height))

    return image, keypoints, occluded_keypoints

In [17]:
import os
import cv2
import json

def apply_random_occlusions_to_folder(src_folder, dest_folder, last_img_idx, num_iterations=1):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    image_files = [f for f in os.listdir(src_folder) if f.endswith(".rgb.jpg")]

    for _ in range(num_iterations):
        for image_file in image_files:
            image_path = os.path.join(src_folder, image_file)
            image = cv2.imread(image_path)
            
            json_file = image_file.replace('.rgb.jpg', '.json')
            with open(os.path.join(src_folder, json_file), 'r') as f:
                annotation = json.load(f)

            # Apply random occlusion and receive the occluded keypoints
            occluded_image, _, occluded_keypoints = random_occlusion(image, annotation['keypoints'])
            
            # For each occluded keypoint, set its visibility to 0
            for idx, kp in occluded_keypoints:
                annotation['keypoints'][idx][0][2] = 0  # Set visibility to 0

            last_img_idx += 1
            new_image_name = f"{str(last_img_idx).zfill(6)}.rgb.jpg"
            cv2.imwrite(os.path.join(dest_folder, new_image_name), occluded_image)
            
            annotation["id"] = last_img_idx
            annotation["image_rgb"] = new_image_name
            new_json_name = f"{str(last_img_idx).zfill(6)}.json"
            with open(os.path.join(dest_folder, new_json_name), 'w') as f:
                json.dump(annotation, f)

    print(f"Processed {len(image_files) * num_iterations} images and saved them in {dest_folder}.")

# Example usage
src_folder = "/home/jc-merlab/Pictures/Data/occ_sim_append/"
dest_folder = "/home/jc-merlab/Pictures/Data/source_folder_occlusion/"
last_img_idx = 1330
apply_random_occlusions_to_folder(src_folder, dest_folder, last_img_idx)

Processed 1331 images and saved them in /home/jc-merlab/Pictures/Data/source_folder_occlusion/.


In [1]:
# def get_random_patch(secondary_folder, occlusion_size):
#     # Randomly choose an image from the secondary folder
#     image_files = [f for f in os.listdir(secondary_folder) if f.endswith((".jpg", ".jpeg", ".png"))]
#     random_image_path = os.path.join(secondary_folder, random.choice(image_files))
    
#     image = cv2.imread(random_image_path)
#     height, width, _ = image.shape
    
#     # Choose a random x, y starting point for the patch
#     x = random.randint(0, width - occlusion_size)
#     y = random.randint(0, height - occlusion_size)
    
#     return image[y:y+occlusion_size, x:x+occlusion_size, :]

In [1]:
import os
import cv2
import random

def get_random_patch(secondary_folder, occlusion_width, occlusion_height):
    # Randomly choose an image from the secondary folder
    image_files = [f for f in os.listdir(secondary_folder) if f.endswith((".jpg", ".jpeg", ".png"))]
    random_image_path = os.path.join(secondary_folder, random.choice(image_files))
    
    image = cv2.imread(random_image_path)
    height, width, _ = image.shape
    
    # Ensure the starting point for the patch won't go out of the image boundaries
    x = random.randint(0, width - occlusion_width)
    y = random.randint(0, height - occlusion_height)
    
    return image[y:y+occlusion_height, x:x+occlusion_width, :]

In [2]:
# import random
# import numpy as np

# def apply_rectangle_occlusion_with_patch(image, x, y, occlusion_size, color=None):
#     patch = get_random_patch(secondary_folder, occlusion_size)
#     image[y:y+occlusion_size, x:x+occlusion_size, :] = patch
#     return image

# def apply_circle_occlusion_with_patch(image, x, y, occlusion_size, secondary_folder):
#     patch = get_random_patch(secondary_folder, occlusion_size)
    
#     mask = np.zeros((occlusion_size, occlusion_size), dtype=np.uint8)
#     center = (occlusion_size // 2, occlusion_size // 2)
#     cv2.circle(mask, center, occlusion_size // 2, (255), -1)
    
#     roi = image[y:y+occlusion_size, x:x+occlusion_size]
#     np.copyto(roi, patch, where=mask[:,:,None].astype(bool))
#     image[y:y+occlusion_size, x:x+occlusion_size] = roi    
#     return image

# def apply_ellipse_occlusion_with_patch(image, x, y, occlusion_size, secondary_folder):
#     patch = get_random_patch(secondary_folder, occlusion_size)
    
#     mask = np.zeros((occlusion_size, occlusion_size), dtype=np.uint8)
#     center = (occlusion_size // 2, occlusion_size // 2)
#     axes = (occlusion_size // 2, random.randint(occlusion_size // 4, occlusion_size // 2))
#     angle = random.randint(0, 180)
#     cv2.ellipse(mask, center, axes, angle, 0, 360, (255), -1)
    
#     roi = image[y:y+occlusion_size, x:x+occlusion_size]
#     np.copyto(roi, patch, where=mask[:,:,None].astype(bool))
#     image[y:y+occlusion_size, x:x+occlusion_size] = roi    
#     return image

# def random_occlusion(image, keypoints, secondary_folder, max_occlusions=3, min_size=60, max_size=100):
#     height, width, _ = image.shape
#     num_occlusions = random.randint(1, max_occlusions)

#     for _ in range(num_occlusions):
#         # Randomly select a keypoint to occlude
#         kp_set = random.choice(keypoints[:6])
#         kp = random.choice(kp_set)[:2]  # Ignore visibility flag

#         # Random size for the occlusion
#         occlusion_size = random.randint(min_size, max_size)

#         # Center the occlusion on the keypoint
#         x = int(kp[0] - occlusion_size/2)
#         y = int(kp[1] - occlusion_size/2)

#         # Ensure the occlusion is within image boundaries
#         x = max(0, min(x, width - occlusion_size))
#         y = max(0, min(y, height - occlusion_size))

#         # Random shape of occlusion
#         shapes = ['rectangle', 'circle', 'ellipse']
#         shape = random.choice(shapes)
#         if shape == 'rectangle':
#             image = apply_rectangle_occlusion_with_patch(image, x, y, occlusion_size, secondary_folder)
#         elif shape == 'circle':
#             image = apply_circle_occlusion_with_patch(image, x, y, occlusion_size, secondary_folder)
#         elif shape == 'ellipse':
#             image = apply_ellipse_occlusion_with_patch(image, x, y, occlusion_size, secondary_folder)

#     return image, keypoints


In [3]:
import random
import numpy as np
import cv2
import math

def tilt_patch(patch):
    angle = random.uniform(-30, 30)  # Tilt in the range [-30, 30] degrees
    height, width, _ = patch.shape
    
    # Get the rotation matrix
    M = cv2.getRotationMatrix2D((width // 2, height // 2), angle, 1)
    
    # Rotate the patch
    rotated_patch = cv2.warpAffine(patch, M, (width, height), borderMode=cv2.BORDER_REPLICATE)
    
    cv2.imshow("rotated_patch", rotated_patch)
    cv2.waitKey(1)
    
    return rotated_patch

# def tilt_patch(patch):
#     angle = random.uniform(-30, 30)  # Tilt in the range [-30, 30] degrees
#     height, width, _ = patch.shape
    
#     # Get the rotation matrix
#     M = cv2.getRotationMatrix2D((width // 2, height // 2), angle, 1)
    
#     # Rotate the patch
#     rotated_patch = cv2.warpAffine(patch, M, (width, height), borderMode=cv2.BORDER_REPLICATE)
    
#     # Recrop to ensure straight edges
#     margin_h = int(0.1 * height)  # Adjust these margins if needed
#     margin_w = int(0.1 * width)
#     cropped_patch = rotated_patch[margin_h:-margin_h, margin_w:-margin_w]
    
#     return cropped_patch

def apply_rectangle_occlusion_with_patch(image, x, y, width, height, patch):
    resized_patch = cv2.resize(patch, (width, height))
    image[y:y+height, x:x+width, :] = resized_patch
    return image

def apply_circle_occlusion_with_patch(image, x, y, occlusion_width, occlusion_height, patch):
    mask = np.zeros((occlusion_height, occlusion_width), dtype=np.uint8)
    
    # Determine the center and size of the circle
    center = (occlusion_width // 2, occlusion_height // 2)
    size = min(occlusion_width, occlusion_height)  # Ensure the circle fits within the ROI
    
    cv2.circle(mask, center, size // 2, (255), -1)

    roi = image[y:y+occlusion_height, x:x+occlusion_width]
    
    # Resize the patch to match the shape of the roi
    patch = cv2.resize(patch, (roi.shape[1], roi.shape[0]))
    
    np.copyto(roi, patch, where=mask[:,:,None].astype(bool))
    image[y:y+occlusion_height, x:x+occlusion_width] = roi
    
    return image

def apply_ellipse_occlusion_with_patch(image, x, y, occlusion_width, occlusion_height, patch):
    mask = np.zeros((occlusion_height, occlusion_width), dtype=np.uint8)
    
    # This calculates the center and axes of the ellipse
    center = (occlusion_width // 2, occlusion_height // 2)
    axes = (occlusion_width // 2, occlusion_height // 2)
    
    angle = random.randint(0, 180)
    cv2.ellipse(mask, center, axes, angle, 0, 360, (255), -1)

    roi = image[y:y+occlusion_height, x:x+occlusion_width]
    
    # Resize the patch to match the shape of the roi
    patch = cv2.resize(patch, (roi.shape[1], roi.shape[0]))
    
    np.copyto(roi, patch, where=mask[:,:,None].astype(bool))
    image[y:y+occlusion_height, x:x+occlusion_width] = roi
    
    return image


# def random_occlusion(image, keypoints, secondary_folder, max_occlusions=3, min_size=60, max_size=100):
#     height, width, _ = image.shape
#     num_occlusions = random.randint(1, max_occlusions)

#     for _ in range(num_occlusions):
#         # Randomly select a keypoint to occlude
#         kp_set = random.choice(keypoints[:6])
#         kp = random.choice(kp_set)[:2]  # Ignore visibility flag

#         # Random size for the occlusion
#         occlusion_width = random.randint(min_size, max_size)
#         occlusion_height = random.randint(min_size, max_size)

#         # Center the occlusion on the keypoint
#         x = int(kp[0] - occlusion_width/2)
#         y = int(kp[1] - occlusion_height/2)

#         # Ensure the occlusion is within image boundaries
#         x = max(0, min(x, width - occlusion_width))
#         y = max(0, min(y, height - occlusion_height))

#         # Get the patch and optionally tilt it
#         patch = get_random_patch(secondary_folder, occlusion_width, occlusion_height)
#         if random.choice([True, False]):  # 50% chance to apply tilt
#             patch = tilt_patch(patch)

#         # Random shape of occlusion
#         shapes = ['rectangle', 'circle', 'ellipse']
#         shape = random.choice(shapes)

#         # Apply the (possibly tilted) patch as occlusion
#         if shape == 'rectangle':
#             image = apply_rectangle_occlusion_with_patch(image, x, y, occlusion_width, occlusion_height, patch)
#         elif shape == 'circle':
#             image = apply_circle_occlusion_with_patch(image, x, y, occlusion_width, occlusion_height, patch)
#         elif shape == 'ellipse':
#             image = apply_ellipse_occlusion_with_patch(image, x, y, occlusion_width, occlusion_height, patch)

#     return image, keypoints


def random_occlusion(image, keypoints, secondary_folder, max_occlusions=3, min_size=60, max_size=100):
    height, width, _ = image.shape
    num_occlusions = random.randint(1, max_occlusions)

    for _ in range(num_occlusions):
        # Randomly select two keypoints
        kp_set_1 = random.choice(keypoints[:6])
        kp1 = random.choice(kp_set_1)[:2]

        kp_set_2 = random.choice(keypoints[:6])
        kp2 = random.choice(kp_set_2)[:2]
        
        # Determine a region in between the two keypoints for the occlusion
        center_x = (kp1[0] + kp2[0]) / 2
        center_y = (kp1[1] + kp2[1]) / 2

        # Random size for the occlusion
        occlusion_width = random.randint(min_size, max_size)
        occlusion_height = random.randint(min_size, max_size)

        # Center the occlusion on the in-between point
        x = int(center_x - occlusion_width/2)
        y = int(center_y - occlusion_height/2)

        # Ensure the occlusion is within image boundaries
        x = max(0, min(x, width - occlusion_width))
        y = max(0, min(y, height - occlusion_height))

        # Get the patch and optionally tilt it
        patch = get_random_patch(secondary_folder, occlusion_width, occlusion_height)
        if random.choice([True, False]):  # 50% chance to apply tilt
            patch = tilt_patch(patch)

        # Random shape of occlusion
        shapes = ['rectangle', 'circle', 'ellipse']
        shape = random.choice(shapes)

        # Apply the (possibly tilted) patch as occlusion
        if shape == 'rectangle':
            image = apply_rectangle_occlusion_with_patch(image, x, y, occlusion_width, occlusion_height, patch)
        elif shape == 'circle':
            image = apply_circle_occlusion_with_patch(image, x, y, occlusion_width, occlusion_height, patch)
        elif shape == 'ellipse':
            image = apply_ellipse_occlusion_with_patch(image, x, y, occlusion_width, occlusion_height, patch)

    return image, keypoints

In [4]:
# import os
# import cv2
# import json

# def apply_random_occlusions_to_folder(src_folder, dest_folder, secondary_folder, last_img_idx):
#     """
#     Apply random occlusions to images in the source folder and save results in the destination folder.

#     src_folder: Path to the folder containing source images and JSON annotations.
#     dest_folder: Path to the folder where occluded images and updated JSON annotations will be saved.
#     last_img_idx: Index of the last image in the source folder.
#     """
    
#     # Ensure the destination folder exists
#     if not os.path.exists(dest_folder):
#         os.makedirs(dest_folder)

#     # List all RGB images in the source folder
#     image_files = [f for f in os.listdir(src_folder) if f.endswith(".rgb.jpg")]
    
#     for image_file in image_files:
#         # Load the image
#         image_path = os.path.join(src_folder, image_file)
#         image = cv2.imread(image_path)
        
#         # Load the corresponding JSON annotation
#         json_file = image_file.replace('.rgb.jpg', '.json')
#         with open(os.path.join(src_folder, json_file), 'r') as f:
#             annotation = json.load(f)

#         # Apply random occlusion to the image using the provided keypoints
#         occluded_image, _ = random_occlusion(image, annotation['keypoints'], secondary_folder)
        
#         # Save the occluded image with a new filename
#         last_img_idx += 1
#         new_image_name = f"{str(last_img_idx).zfill(6)}.rgb.jpg"
#         cv2.imwrite(os.path.join(dest_folder, new_image_name), occluded_image)
        
#         # Update the JSON annotation with the new filename and save it
#         annotation["id"] = last_img_idx  # Update the id field to match the filename
#         annotation["image_rgb"] = new_image_name
#         new_json_name = f"{str(last_img_idx).zfill(6)}.json"
#         with open(os.path.join(dest_folder, new_json_name), 'w') as f:
#             json.dump(annotation, f)

#     print(f"Processed {len(image_files)} images and saved them in {dest_folder}.")


# # Example usage:
# secondary_folder = "/home/jc-merlab/Downloads/unlabeled2017/"
# src_folder = "/home/jc-merlab/Pictures/Data/occlusion_append/"
# dest_folder = "/home/jc-merlab/Pictures/Data/source_folder_occlusion/"
# last_img_idx = 2314
# apply_random_occlusions_to_folder(src_folder, dest_folder, secondary_folder, last_img_idx)


In [5]:
import os
import cv2
import json

def apply_random_occlusions_to_folder(src_folder, dest_folder, secondary_folder, last_img_idx, num_iterations=5):
    """
    Apply random occlusions multiple times to images in the source folder and save results in the destination folder.

    src_folder: Path to the folder containing source images and JSON annotations.
    dest_folder: Path to the folder where occluded images and updated JSON annotations will be saved.
    secondary_folder: Path to the folder with additional data, used in random_occlusion.
    last_img_idx: Index of the last image in the source folder.
    num_iterations: Number of times occlusions are applied to each image.
    """
    
    # Ensure the destination folder exists
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    # List all RGB images in the source folder
    image_files = [f for f in os.listdir(src_folder) if f.endswith(".rgb.jpg")]

    for _ in range(num_iterations):
        for image_file in image_files:
            # Load the image
            image_path = os.path.join(src_folder, image_file)
            image = cv2.imread(image_path)
            
            # Load the corresponding JSON annotation
            json_file = image_file.replace('.rgb.jpg', '.json')
            with open(os.path.join(src_folder, json_file), 'r') as f:
                annotation = json.load(f)

            # Apply random occlusion to the image using the provided keypoints
            occluded_image, _ = random_occlusion(image, annotation['keypoints'], secondary_folder)
            
            # Save the occluded image with a new filename
            last_img_idx += 1
            new_image_name = f"{str(last_img_idx).zfill(6)}.rgb.jpg"
            cv2.imwrite(os.path.join(dest_folder, new_image_name), occluded_image)
            
            # Update the JSON annotation with the new filename and save it
            annotation["id"] = last_img_idx  # Update the id field to match the filename
            annotation["image_rgb"] = new_image_name
            new_json_name = f"{str(last_img_idx).zfill(6)}.json"
            with open(os.path.join(dest_folder, new_json_name), 'w') as f:
                json.dump(annotation, f)

    print(f"Processed {len(image_files) * num_iterations} images and saved them in {dest_folder}.")

# Example usage:
secondary_folder = "/home/jc-merlab/Downloads/unlabeled2017/"
src_folder = "/home/jc-merlab/Pictures/Data/occlusion_append/"
dest_folder = "/home/jc-merlab/Pictures/Data/source_folder_occlusion/"
last_img_idx = 2314
apply_random_occlusions_to_folder(src_folder, dest_folder, secondary_folder, last_img_idx)

Processed 2315 images and saved them in /home/jc-merlab/Pictures/Data/source_folder_occlusion/.
